In [ ]:
# 작업자 : 권은이

import json
from pathlib import Path
from tqdm import tqdm
from rapidfuzz import fuzz


def normalize_name(name: str):
    if not name:
        return ""
    import re
    name = name.lower()
    # 문자/숫자/한글만 남기기
    name = re.sub(r"[^\w가-힣]", "", name)
    return name.strip()


def dedupe_clubs_smart(
    input_path,
    output_path,
    name_extract_fn,
    similarity_threshold=85,
):
    input_path = Path(input_path)
    output_path = Path(output_path)

    # 1) 전체 데이터 로드 + 동아리 이름 추출
    data = []
    with input_path.open("r", encoding="utf-8") as fin:
        for line in fin:
            line = line.strip()
            if not line:
                continue
            try:
                item = json.loads(line)
            except:
                continue

            text = item.get("all_text", "")
            club_name = name_extract_fn(text)
            club_name_norm = normalize_name(club_name)

            data.append({
                "raw": item,
                "club_name": club_name,
                "club_name_norm": club_name_norm,
                "text_len": len(text),
            })

    print(f"총 {len(data)}개 로드됨")

    # 2) 이름 유사도 기준으로 클러스터링
    clusters = []
    for item in tqdm(data, desc="Clustering"):
        name = item["club_name_norm"]

        if not name:
            # 이름 못 뽑은 애들은 그냥 단독 클러스터
            clusters.append([item])
            continue

        matched = False
        for cluster in clusters:
            rep = cluster[0]["club_name_norm"]
            sim = fuzz.partial_ratio(name, rep)  # 0~100

            if sim >= similarity_threshold:
                cluster.append(item)
                matched = True
                break

        if not matched:
            clusters.append([item])

    print(f"동아리 그룹 수: {len(clusters)}")

    # 3) 각 그룹에서 "가장 긴 글" 하나만 선택
    deduped = []
    for cluster in clusters:
        best = max(cluster, key=lambda x: x["text_len"])
        deduped.append(best["raw"])

    print(f"최종 남은 글 수: {len(deduped)}")

    # 4) 저장
    with output_path.open("w", encoding="utf-8") as fout:
        for item in deduped:
            fout.write(json.dumps(item, ensure_ascii=False) + "\n")

    print("완료!")


# ==========================
# 실제 실행
# ==========================

dedupe_clubs_smart(
    input_path=Path("/content/drive/MyDrive/Colab Notebooks/NLP/everytime_crawling_club.jsonl"),
    output_path=Path("/content/drive/MyDrive/Colab Notebooks/NLP/everytime_crawling_club_dedup_smart.jsonl"),
    name_extract_fn=extract_club_name,
)

총 1877개 로드됨


Clustering: 100%|██████████| 1877/1877 [00:04<00:00, 390.80it/s]


동아리 그룹 수: 537
최종 남은 글 수: 537
완료!


In [1]:
import json
import re
import html
from pathlib import Path
from tqdm import tqdm

# ==============================
# 0. 입력/출력 파일 경로 설정
# ==============================
INPUT_JSONL  = "/content/drive/MyDrive/Colab Notebooks/NLP/everytime_crawling_club_dedup_smart.jsonl"
OUTPUT_JSONL = "/content/drive/MyDrive/Colab Notebooks/NLP/everytime_club_parsed.jsonl"


# ==============================
# 1. HTML에서 제목 / 본문 추출
# ==============================

TAG_RE = re.compile(r"<[^>]+>")  # 태그 제거용

def strip_tags(text: str) -> str:
    """HTML 태그 제거 + 공백 정리."""
    text = TAG_RE.sub("", text)
    text = html.unescape(text)
    # 줄바꿈/공백 정리
    lines = [l.strip() for l in text.splitlines()]
    lines = [l for l in lines if l]
    return "\n".join(lines).strip()

def extract_title_and_body_from_html(all_html: str):
    """
    all_html 에서
      - 첫 번째 <h2 ...>...</h2> 를 제목으로
      - 그 뒤에 나오는 첫 번째 <p class="large"> ... </p> 를 본문으로 사용
    """
    if not all_html:
        return "", ""

    # 1) 제목: 첫 번째 <h2 ...>...</h2>
    h2_match = re.search(r"<h2[^>]*>(.*?)</h2>", all_html, flags=re.DOTALL | re.IGNORECASE)
    if h2_match:
        raw_title = h2_match.group(1)
        title = strip_tags(raw_title)
    else:
        title = ""

    body = ""

    # 2) 본문: 위 h2 바로 뒤의 <p class="large">...</p> 블록
    if h2_match:
        start_pos = h2_match.end()
        rest_html = all_html[start_pos:]
        p_match = re.search(r"<p[^>]*class=\"large\"[^>]*>(.*?)</p>", rest_html,
                            flags=re.DOTALL | re.IGNORECASE)
        if p_match:
            raw_body = p_match.group(1)
            body = strip_tags(raw_body)
        else:
            # fallback: h2 이후 전체에서 태그만 제거해서 본문으로 사용
            body = strip_tags(rest_html)
    else:
        # h2가 없으면 전체를 태그 제거해서 본문으로 사용
        body = strip_tags(all_html)

    return title, body


# ==============================
# 2. 교내 / 연합 분류 (룰 기반)
# ==============================
def classify_scope(title: str, body: str) -> str:
    """
    교내 / 연합 여부를 간단한 키워드 규칙으로 분류
    return: '교내', '연합', '미상'
    """
    txt = (title + "\n" + body).lower()

    # 연합 동아리 키워드 우선
    union_keywords = [
        "연합동아리", "연합 동아리", "연합 소모임",
        "대학연합", "대학 연합", "연합 사진동아리",
        "대학 연합 사진동아리"
    ]
    if any(k in txt for k in union_keywords):
        return "연합"

    # '연합' + '동아리' 같이 등장하는 경우도 연합으로 처리
    if "연합" in txt and "동아리" in txt:
        return "연합"

    # 교내 동아리 쪽 키워드
    campus_keywords = [
        "중앙동아리", "중앙 동아리",
        "교내 동아리", "교내동아리",
        "숭실대", "숭실대학교"
    ]
    if any(k in txt for k in campus_keywords):
        return "교내"

    return "미상"


# ==============================
# 3. 동아리 종류 분류 (대분류 / 소분류, 룰 기반)
# ==============================
def classify_category(title: str, body: str):
    """
    동아리 종류를 간단한 키워드 규칙으로 분류
    return: (main_category, sub_category)
    """
    txt = (title + "\n" + body).lower()

    # ------------------
    # 1) 운동 관련
    # ------------------
    sports_pairs = [
        ("축구", "운동", "축구/풋살"),
        ("풋살", "운동", "축구/풋살"),
        ("농구", "운동", "농구"),
        ("배드민턴", "운동", "배드민턴"),
        ("테니스", "운동", "테니스"),
        ("탁구", "운동", "탁구"),
        ("헬스", "운동", "헬스/피트니스"),
        ("러닝", "운동", "러닝"),
    ]
    for kw, main, sub in sports_pairs:
        if kw in txt:
            return main, sub

    # ------------------
    # 2) 사진 / 영상 / 예능 콘텐츠
    # ------------------
    photo_keywords = [
        "사진동아리", "사진 동아리", "사진", "출사",
        "카메라", "dslr", "미러리스", "필카", "필름카메라"
    ]
    if any(k in txt for k in photo_keywords):
        return "취미/예술", "사진"

    entertainment_keywords = [
        "예능", "리얼버라이어티", "버라이어티",
        "콘텐츠", "컨텐츠",
        "유튜브", "youtube", "넷플릭스",
        "방송국", "방송 동아리"
    ]
    if any(k in txt for k in entertainment_keywords):
        return "취미/예술", "예능/콘텐츠"

    video_keywords = [
        "영상제작", "영상 제작", "촬영 편집", "영상 편집",
        "videography", "film making"
    ]
    if any(k in txt for k in video_keywords):
        return "취미/예술", "영상/미디어"

    # ------------------
    # 3) 독서 / 글쓰기 / 인문
    # ------------------
    reading_keywords = [
        "독서동아리", "독서 동아리", "독서모임",
        "북클럽", "책 모임", "책읽기", "독서"
    ]
    if any(k in txt for k in reading_keywords):
        return "학술/독서", "독서"

    writing_keywords = [
        "글쓰기", "에세이", "시 쓰기", "소설 쓰기", "창작 동아리"
    ]
    if any(k in txt for k in writing_keywords):
        return "학술/독서", "글쓰기/창작"

    # ------------------
    # 4) 음악 / 공연
    # ------------------
    music_keywords = [
        "밴드", "악기", "기타동아리", "기타 동아리",
        "보컬", "합창", "공연동아리", "버스킹",
        "오케스트라", "연주회"
    ]
    if any(k in txt for k in music_keywords):
        return "취미/예술", "음악/공연"

    # ------------------
    # 5) 봉사 / 사회
    # ------------------
    volunteer_keywords = [
        "봉사동아리", "봉사 동아리", "봉사활동",
        "재능기부", "사회공헌", "사회 공헌"
    ]
    if any(k in txt for k in volunteer_keywords):
        return "봉사", "봉사"

    # ------------------
    # 6) 스터디 / 학술
    # ------------------
    study_keywords = [
        "스터디", "학술동아리", "학술 동아리",
        "전공학회", "학회", "고시", "공모전", "세미나"
    ]
    if any(k in txt for k in study_keywords):
        return "학술/스터디", "학술/전공"

    # ------------------
    # 7) 기타 취미 (보드게임, 방탈출 등)
    # ------------------
    hobby_keywords = [
        "보드게임", "보드 게임", "방탈출",
        "게임 동아리", "게임동아리", "ttrpg"
    ]
    if any(k in txt for k in hobby_keywords):
        return "취미/기타", "게임/보드게임"

    # 기본값
    return "기타", "기타"


# ==============================
# 4. 전체 파이프라인
# ==============================
def process_file(input_path: str, output_path: str):
    in_path = Path(input_path)
    out_path = Path(output_path)

    with in_path.open("r", encoding="utf-8") as f_in, \
         out_path.open("w", encoding="utf-8") as f_out:

        for line in tqdm(f_in, desc="Parsing posts (HTML)"):
            line = line.strip()
            if not line:
                continue

            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                continue

            all_html = obj.get("all_html", "")

            # 1) HTML에서 제목 / 본문 분리
            title, body = extract_title_and_body_from_html(all_html)

            # 2) 교내 / 연합 분류
            scope = classify_scope(title, body)

            # 3) 동아리 종류 분류
            main_cat, sub_cat = classify_category(title, body)

            # 원본 obj 에 필드 추가
            obj["title"] = title            # HTML <h2>에서 뽑은 제목
            obj["body"] = body              # HTML <p class="large">에서 뽑은 내용
            obj["scope"] = scope            # 교내 / 연합 / 미상
            obj["main_category"] = main_cat # 대분류
            obj["sub_category"] = sub_cat   # 소분류

            f_out.write(json.dumps(obj, ensure_ascii=False) + "\n")


if __name__ == "__main__":
    process_file(INPUT_JSONL, OUTPUT_JSONL)

Parsing posts (HTML): 537it [00:01, 459.77it/s]


In [4]:
import pandas as pd

df = pd.read_json(
    "/content/drive/MyDrive/Colab Notebooks/NLP/everytime_club_parsed.jsonl",
    lines=True
)
df[["main_category","sub_category"]].head()

,main_category,sub_category
0,취미/예술,사진
1,취미/예술,예능/콘텐츠
2,학술/독서,독서
3,학술/독서,독서
4,기타,기타
